In [1]:
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("../src/")
%pwd

'/home/jovyan/nfs-home/scalable_ml_pipelines/src'

In [3]:
import socket

LOCAL_IP = socket.gethostbyname(socket.gethostname())

In [4]:
name_space = "eabraham-373705"

# Master node
kubernetes_master_url = "k8s://https://10.32.7.103:6443"

# Resource settings
driver_cores = "8"
executor_cores = "8"
driver_memory = "30g"
executor_memory = "30g"
executor_memory_overhead = "2g"

# These are the limits
cpu_limit = "3"  # 12 cores
memory_limit = "32g"  # Upto 32 GB
executor_limit = "8"

In [5]:
from pyspark.sql import SparkSession

APP_NAME = 'scalables_executor'


spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master(kubernetes_master_url)\
    .config("spark.driver.host", LOCAL_IP)\
    .config("spark.driver.bindAddress", "0.0.0.0")\
    .config("spark.executor.instances", "2")\
    .config("spark.executor.cores", executor_cores)\
    .config("spark.executor.memory", executor_memory)\
    .config("spark.memory.fraction", "0.8")\
    .config("spark.memory.storageFraction", "0.2")\
    .config("spark.kubernetes.executor.limit.cores", executor_limit)\
    .config("spark.kubernetes.namespace", name_space)\
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")\
    .config("spark.kubernetes.driver.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.deleteOnTermination", "false") \
    .config("spark.kubernetes.container.image.pullPolicy", "Always") \
    .config("spark.kubernetes.container.image", "node03.st:5000/pyspark-hdfs-jupyter:eabraham-373705-v4-executor")\
    .config("spark.local.dir", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .getOrCreate()


23/12/18 18:35:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/18 18:35:26 WARN spark.SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [6]:
spark

In [7]:
from us_used_cars_ml_pipeline.config.configuration import ConfigurationManager
from us_used_cars_ml_pipeline.components.stacking_regressor_modeling import StackingRegressorModeling

config_manager = ConfigurationManager()
stacking_regressor_modeling_config = config_manager.get_stacking_regressor_modeling_config()

stacking_regressor_modeling = StackingRegressorModeling(stacking_regressor_modeling_config)

[2023-12-18 18:35:34,108: 145: numexpr.utils: INFO: utils:  Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2023-12-18 18:35:34,109: 157: numexpr.utils: INFO: utils:  NumExpr defaulting to 8 threads.]
[2023-12-18 18:35:34,759: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/config/config.yaml loaded successfully]
[2023-12-18 18:35:34,763: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/params.yaml loaded successfully]
[2023-12-18 18:35:34,766: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/schema.yaml loaded successfully]


In [8]:
stacking_regressor_modeling_config

StackingRegressorModelingConfig(path_to_prepared_data='hdfs:///home/eabraham-373705/data/prepared/', path_to_importances='us_used_cars_ml_pipeline/tests/importances/', path_to_parameters_grid='us_used_cars_ml_pipeline/constants/grid_search/', path_to_best_params='us_used_cars_ml_pipeline/tests/models_tuning/best_params/CVTuner/', path_to_predictions='hdfs:///home/eabraham-373705/data/predictions', path_to_stacking_models='hdfs:///home/eabraham-373705/models/stacking_models', n_folds=5, seed=42, base_models_names=BoxList(['LinearRegression', 'RandomForestRegressor', 'GBTRegressor']), n_feats=ConfigBox({'LinearRegression': 70, 'RandomForestRegressor': 80, 'GBTRegressor': 70}), meta_model_name='LinearRegression', meta_model_params=ConfigBox({'labelCol': 'price', 'predictionCol': 'prediction', 'featuresCol': 'first_level_predictions', 'maxIter': 100, 'regParam': 1, 'elasticNetParam': 0.5}), tuner='CVTuner', tuner_config=ConfigBox({'metric': 'r2', 'seed': 42, 'n_folds': 5, 'path_to_paramete

In [9]:
%%time
stacking_regressor_modeling.run_stage(spark, is_new_data=False)

[2023-12-17 16:13:02,770: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/grid_search/LinearRegression.yaml loaded successfully]
[2023-12-17 16:13:02,774: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/tests/models_tuning/best_params/CVTuner/LinearRegression.yaml loaded successfully]
[2023-12-17 16:13:02,779: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/grid_search/RandomForestRegressor.yaml loaded successfully]
[2023-12-17 16:13:02,783: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/tests/models_tuning/best_params/CVTuner/RandomForestRegressor.yaml loaded successfully]
[2023-12-17 16:13:02,789: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/grid_search/GBTRegressor.yaml loaded successfully]
[2023-12-17 16:13:02,793: 44: us_used_cars_ml_pipeline_logger: 

[2023-12-17 16:13:06,881: 142: us_used_cars_ml_pipeline_logger: INFO: stacking_regressor_modeling:  Prepared data has been loaded]
[2023-12-17 16:13:07,225: 146: us_used_cars_ml_pipeline_logger: INFO: stacking_regressor_modeling:  Fold column has been created]
[2023-12-17 16:13:07,226: 149: us_used_cars_ml_pipeline_logger: INFO: stacking_regressor_modeling:  PART I. STARTING]


Training base models:   0%|          | 0/3 [00:00<?, ?it/s]

[2023-12-17 16:13:07,286: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/tests/importances/LinearRegression.yaml loaded successfully]


23/12/17 16:13:08 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


...

[2023-12-17 16:18:54,161: 337: us_used_cars_ml_pipeline_logger: INFO: tuners:  {'elasticNetParam': 1, 'regParam': 0.1} -> r2: 0.7783350028681215]


[2023-12-17 16:18:56,993: 337: us_used_cars_ml_pipeline_logger: INFO: tuners:  {'elasticNetParam': 1, 'regParam': 1} -> r2: 0.7783301662123067]


[2023-12-17 16:18:59,849: 337: us_used_cars_ml_pipeline_logger: INFO: tuners:  {'elasticNetParam': 1, 'regParam': 10} -> r2: 0.7782797249467981]


[2023-12-17 16:19:02,826: 337: us_used_cars_ml_pipeline_logger: INFO: tuners:  {'elasticNetParam': 1, 'regParam': 100} -> r2: 0.7775867798578073]
[2023-12-17 16:19:02,846: 66: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file saved at: us_used_cars_ml_pipeline/tests/meta_model_tuning/scores/LinearRegression.yaml]
[2023-12-17 16:19:02,847: 347: us_used_cars_ml_pipeline_logger: INFO: tuners:  Scores for LinearRegression model have been saved]
[2023-12-17 16:19:02,851: 66: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file saved at: us_used_cars_ml_pipeline/tests/meta_model_tuning/best_params/LinearRegression.yaml]
[2023-12-17 16:19:02,852: 351: us_used_cars_ml_pipeline_logger: INFO: tuners:  Best set of parameters for LinearRegression model has been saved]
[2023-12-17 16:19:02,853: 186: us_used_cars_ml_pipeline_logger: INFO: stacking_regressor_modeling:  PART VII. COMPLETED]
[2023-12-17 16:19:02,854: 189: us_used_cars_ml_pipeline_logger: INFO: stacking_regressor_modeling: 

[2023-12-17 16:19:07,332: 191: us_used_cars_ml_pipeline_logger: INFO: stacking_regressor_modeling:  PART VIII. COMPLETED]
CPU times: user 2.35 s, sys: 776 ms, total: 3.12 s
Wall time: 6min 4s
